# Rf 3 1 6_Llratioplot
Multidimensional models: using the likelihood ratio technique to construct a signal enhanced one-dimensional
projection of a multi-dimensional p.d.f.



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:51 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "RooPolynomial.h"
#include "RooAddPdf.h"
#include "RooProdPdf.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Create 3d pdf and data
 -------------------------------------------

Create observables

In [3]:
RooRealVar x("x", "x", -5, 5);
RooRealVar y("y", "y", -5, 5);
RooRealVar z("z", "z", -5, 5);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Create signal pdf gauss(x)*gauss(y)*gauss(z)

In [4]:
RooGaussian gx("gx", "gx", x, RooConst(0), RooConst(1));
RooGaussian gy("gy", "gy", y, RooConst(0), RooConst(1));
RooGaussian gz("gz", "gz", z, RooConst(0), RooConst(1));
RooProdPdf sig("sig", "sig", RooArgSet(gx, gy, gz));

Create background pdf poly(x)*poly(y)*poly(z)

In [5]:
RooPolynomial px("px", "px", x, RooArgSet(RooConst(-0.1), RooConst(0.004)));
RooPolynomial py("py", "py", y, RooArgSet(RooConst(0.1), RooConst(-0.004)));
RooPolynomial pz("pz", "pz", z);
RooProdPdf bkg("bkg", "bkg", RooArgSet(px, py, pz));

Create composite pdf sig+bkg

In [6]:
RooRealVar fsig("fsig", "signal fraction", 0.1, 0., 1.);
RooAddPdf model("model", "model", RooArgList(sig, bkg), fsig);

RooDataSet *data = model.generate(RooArgSet(x, y, z), 20000);

Project pdf and data on x
 -------------------------------------------------

Make plain projection of data and pdf on x observable

In [7]:
RooPlot *frame = x.frame(Title("Projection of 3D data and pdf on X"), Bins(40));
data->plotOn(frame);
model.plotOn(frame);

[#1] INFO:Plotting -- RooAbsReal::plotOn(model) plot on x integrates over variables (y,z)


Define projected signal likelihood ratio
 ----------------------------------------------------------------------------------

Calculate projection of signal and total likelihood on (y,z) observables
 i.e. integrate signal and composite model over x

In [8]:
RooAbsPdf *sigyz = sig.createProjection(x);
RooAbsPdf *totyz = model.createProjection(x);

Construct the log of the signal / signal+background probability

In [9]:
RooFormulaVar llratio_func("llratio", "log10(@0)-log10(@1)", RooArgList(*sigyz, *totyz));

Plot data with a llratio cut
 -------------------------------------------------------

Calculate the llratio value for each event in the dataset

In [10]:
data->addColumn(llratio_func);

Extract the subset of data with large signal likelihood

In [11]:
RooDataSet *dataSel = (RooDataSet *)data->reduce(Cut("llratio>0.7"));

Make plot frame

In [12]:
RooPlot *frame2 = x.frame(Title("Same projection on X with LLratio(y,z)>0.7"), Bins(40));

Plot select data on frame

In [13]:
dataSel->plotOn(frame2);

Make mc projection of pdf with same llratio cut
 ---------------------------------------------------------------------------------------------

Generate large number of events for mc integration of pdf projection

In [14]:
RooDataSet *mcprojData = model.generate(RooArgSet(x, y, z), 10000);

Calculate ll ratio for each generated event and select mc events with llratio)0.7

In [15]:
mcprojData->addColumn(llratio_func);
RooDataSet *mcprojDataSel = (RooDataSet *)mcprojData->reduce(Cut("llratio>0.7"));

Project model on x, integrating projected observables (y,z) with monte carlo technique
 on set of events with the same llratio cut as was applied to data

In [16]:
model.plotOn(frame2, ProjWData(*mcprojDataSel));

TCanvas *c = new TCanvas("rf316_llratioplot", "rf316_llratioplot", 800, 400);
c->Divide(2);
c->cd(1);
gPad->SetLeftMargin(0.15);
frame->GetYaxis()->SetTitleOffset(1.4);
frame->Draw();
c->cd(2);
gPad->SetLeftMargin(0.15);
frame2->GetYaxis()->SetTitleOffset(1.4);
frame2->Draw();

[#1] INFO:Plotting -- RooAbsReal::plotOn(model) plot on x averages using data variables (y,z)
[#1] INFO:Plotting -- RooAbsReal::plotOn(model) only the following components of the projection data will be used: (y,z)
[#1] INFO:Plotting -- RooDataWeightedAverage::ctor(modelDataWgtAvg) constructing data weighted average of function model_Norm[x] over 767 data points of (y,z) with a total weight of 767
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (gy,gz,py,pz)
.................................................................................................................................................

Draw all canvases 

In [17]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()